# 교재 7.1 예제

주제: 블록화

엔지니어는 절삭속도(A), 공구기하학(B), 절삭각(C)이 공구수명(시간당)에 미치는 효과에 대해 관심이 있다. 각 요소들은 두 개의 수준을 유지한다.

각 __I,II,II__ 는 블록이다.

|A|B|C|처리조합|I|II|III|
|---|---|---|---|---|---|---|
|-|-|-|(1)|22|31|25|
|+|-|-|a|32|43|29|
|-|+|-|b|35|34|50|
|+|+|-|ab|55|47|46|
|-|-|+|c|44|45|38|
|+|-|+|ac|40|37|36|
|-|+|+|bc|60|50|54|
|+|+|+|abc|39|41|47|

# (a) 요인 효과 추정

설계행렬의 부호가 +인 것끼리 더하고 -인 것끼리 더해서 빼면 된다.

## 데이터프레임 설정

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.DataFrame({
    'a': [-1, 1, -1, 1, -1, 1, -1, 1],
    'b': [-1, -1, 1, 1, -1, -1, 1, 1],
    'c': [-1,-1,-1,-1, 1, 1, 1, 1],
    '처리조합': ['(1)', 'a', 'b', 'ab', 'c', 'ac', 'bc', 'abc'],
    'I': [22, 32, 35, 55, 44, 40, 60, 39],
    'II': [31, 43, 34, 47, 45, 37, 50, 41],
    'III': [25, 29, 50, 46, 38, 36, 54, 47]
})

df

,a,b,c,처리조합,I,II,III
0,-1,-1,-1,(1),22,31,25
1,1,-1,-1,a,32,43,29
2,-1,1,-1,b,35,34,50
3,1,1,-1,ab,55,47,46
4,-1,-1,1,c,44,45,38
5,1,-1,1,ac,40,37,36
6,-1,1,1,bc,60,50,54
7,1,1,1,abc,39,41,47


In [2]:
from DOE import effect_est

temp = dict()
for fac in df['처리조합']:
    if fac == '(1)':
        continue
    temp['Effect of '+fac] = effect_est(data=df, sign_col=[f for f in fac], value_cols=['I','II','III'])
pd.Series(temp)

Effect of a       0.50
Effect of b      17.00
Effect of ab     -2.50
Effect of c      10.25
Effect of ac    -13.25
Effect of bc     -4.25
Effect of abc    -3.25
dtype: float64

# 유의수준 0.05 하 분산분석

In [3]:
df = df.melt(id_vars=['a','b','c','처리조합'], value_vars=['I','II','III'], var_name='블록', value_name='Y').copy()
df

,a,b,c,처리조합,블록,Y
0,-1,-1,-1,(1),I,22
1,1,-1,-1,a,I,32
2,-1,1,-1,b,I,35
3,1,1,-1,ab,I,55
4,-1,-1,1,c,I,44
5,1,-1,1,ac,I,40
6,-1,1,1,bc,I,60
7,1,1,1,abc,I,39
8,-1,-1,-1,(1),II,31
9,1,-1,-1,a,II,43


In [4]:
import statsmodels.api as sm

model = sm.OLS.from_formula(formula='Y ~ a * b * c + C(블록)', data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     5.206
Date:                Mon, 02 Dec 2024   Prob (F-statistic):            0.00317
Time:                        14:22:08   Log-Likelihood:                -70.055
No. Observations:                  24   AIC:                             160.1
Df Residuals:                      14   BIC:                             171.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       40.8750      2.075     19.702      0.000      36.425      45.325
C(블록)[T.II]      0.1250      2.934      0.043      0.967      -6.168       6.418
C(블록)[T.III]    -0.2500      2.934     -0.085      0.933      -6.543       6.043
a                0.1667      1.198      0.139      0.891      -2.402       2.736
b                5.6667      1.198      4.731      0.000       3.098       8.236
a:b             -0.8333      1.198     -0.696      0.498      -3.402       1.736
c                3.4167      1.198      2.852      0.013       0.848       5.986
a:c             -4.4167      1.198     -3.687      0.002      -6.986      -1.848
b:c             -1.4167      1.198     -1.183      0.257      -3.986       1.152
a:b:c           -1.0833      1.198     -0.904      0.381      -3.652       1.486
==============================================================================
Omnibus:                        2.526   Durbin-Watson:                   2.208
Prob(Omnibus):                  0.283   Jarque-Bera (JB):                2.169
Skew:                           0.683   Prob(JB):                        0.338
Kurtosis:                       2.451   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(블록),2.0,0.583333,0.291667,0.008470,0.991571
a,1.0,0.666667,0.666667,0.019360,0.891320
b,1.0,770.666667,770.666667,22.380640,0.000322
a:b,1.0,16.666667,16.666667,0.484010,0.497998
c,1.0,280.166667,280.166667,8.136214,0.012789
a:c,1.0,468.166667,468.166667,13.595851,0.002438
b:c,1.0,48.166667,48.166667,1.398790,0.256623
a:b:c,1.0,28.166667,28.166667,0.817978,0.381072
Residual,14.0,482.083333,34.434524,NaN,NaN


블록을 도입함으로써 블록이 설명하는 분산이 오차에서 제거되기 때문에 효과의 유의성이 증가할 수 있다.